In [28]:
# Langchain
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
import openai
import config

In [29]:
api_key = config.api_key
openai.api_key = api_key

In [30]:
url = "https://wttr.in/Thessaloniki?format=3"

In [31]:
loader = WebBaseLoader(url)
raw_docs = loader.load()

In [32]:
text_splitter = RecursiveCharacterTextSplitter()
docs = text_splitter.split_documents(raw_docs)

In [33]:
#text into vectors
# embeddings = OpenAIEmbeddings(openai_api_key = api_key)

from langchain.embeddings import HuggingFaceEmbeddings
# we use instead a free model from HuggingFace (runs locally)
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

In [34]:
vectorstore = FAISS.from_documents(docs, embeddings)

In [35]:
memory = ConversationBufferMemory(memory_key = "chat_history", return_messages=True)

In [36]:
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate

# Optional: force English
system_message = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant that always answers in English."
)

In [37]:
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(openai_api_key = api_key, 
                                                  model = "gpt-4o-mini", 
                                                  temperature = 0), 
                                           vectorstore.as_retriever(), 
                                           memory = memory)


In [38]:
query = "What's the weather in Thessaloniki at the moment?"

In [39]:
#result = qa({"question": query})
result = qa.invoke({"question": query})
print(result["answer"])


The weather in Thessaloniki at the moment is sunny with a temperature of +18°C.


In [40]:
result["answer"]

'The weather in Thessaloniki at the moment is sunny with a temperature of +18°C.'

In [41]:
docs = vectorstore.as_retriever().get_relevant_documents("What is the weather?")
print(docs[0].page_content[:250])


Thessaloniki: ☀️   +18°C
